In [4]:
import gym
from gym import spaces
from gym.utils import seeding

In [6]:
class SnakeEnv(gym.Env):
    metadata = {'render.modes': ['human', 'not_human']}
    def __init__(self):
    # Definitions for game.
    # Board 15 x 15 to start
        self.width = 15
        self.height = 15
        self.squares = self.width*self.height
    # Running score
        self.score = 0
    #Gym implementation members:
        self.action_space = spaces.Discrete(4)obs
    def step(self, action):
        return
    def reset(self):
        return
    def render(self, mode='human', close=False):
        return